1615. Maximal Network Rank
Medium
Topics
premium lock icon
Companies
Hint
There is an infrastructure of n cities with some number of roads connecting these cities. Each roads[i] = [ai, bi] indicates that there is a bidirectional road between cities ai and bi.

The network rank of two different cities is defined as the total number of directly connected roads to either city. If a road is directly connected to both cities, it is only counted once.

The maximal network rank of the infrastructure is the maximum network rank of all pairs of different cities.

Given the integer n and the array roads, return the maximal network rank of the entire infrastructure.

 

Example 1:



Input: n = 4, roads = [[0,1],[0,3],[1,2],[1,3]]
Output: 4
Explanation: The network rank of cities 0 and 1 is 4 as there are 4 roads that are connected to either 0 or 1. The road between 0 and 1 is only counted once.
Example 2:



Input: n = 5, roads = [[0,1],[0,3],[1,2],[1,3],[2,3],[2,4]]
Output: 5
Explanation: There are 5 roads that are connected to cities 1 or 2.
Example 3:

Input: n = 8, roads = [[0,1],[1,2],[2,3],[2,4],[5,6],[5,7]]
Output: 5
Explanation: The network rank of 2 and 5 is 5. Notice that all the cities do not have to be connected.
 

Constraints:

2 <= n <= 100
0 <= roads.length <= n * (n - 1) / 2
roads[i].length == 2
0 <= ai, bi <= n-1
ai != bi
Each pair of cities has at most one road connecting them.

In [ ]:
class Solution:
    def maximalNetworkRank(self, n, roads):
        graph = [[] for _ in range(n)]
        
        for a, b in roads:
            graph[a].append(b)
            graph[b].append(a)
        
        max_rank = 0
        for i in range(n):
            for j in range(i + 1, n):
                rank = len(graph[i]) + len(graph[j])
                if j in graph[i] or i in graph[j]:
                    rank -= 1
                max_rank = max(max_rank, rank)
        
        return max_rank

In [ ]:
class Solution:
    def maximalNetworkRank(self, n: int, roads: List[List[int]]) -> int:
        # Make a adjacency matrix for the graph
        # and store degree count for each vertex
        degrees = [0] * n
        isConnected = [[False] * n for _ in range(n)]
        for road in roads:
            a, b = road[0], road[1]
            degrees[a] += 1
            degrees[b] += 1
            isConnected[a][b] = True
            isConnected[b][a] = True

        # Find out maximum & second maximum degrees in the graph
        maxDegree = 0
        secMaxDegree = 0
        for degree in degrees:
            if degree > maxDegree:
                secMaxDegree = maxDegree
                maxDegree = degree
            elif degree > secMaxDegree and degree != maxDegree:
                secMaxDegree = degree

        # store cities/vertices with max & second max degree count
        maxCities = []
        secMaxCities = []
        for i in range(n):
            if degrees[i] == maxDegree:
                maxCities.append(i)
            elif degrees[i] == secMaxDegree:
                secMaxCities.append(i)

        # if we are having more than 1 cities with max degrees
        if len(maxCities) > 1:
            for i in range(len(maxCities)):
                for j in range(i + 1, len(maxCities)):
                    u, v = maxCities[i], maxCities[j]
                    # If any of 2 max degree cities are not directly connected
                    # then answer is (2 * maxDegree)
                    if not isConnected[u][v]:
                        return 2 * maxDegree

            # If all of them are directly connected
            # then answer is (2 * maxDegree - 1)
            return 2 * maxDegree - 1

        # If there is a single max Degree city
        u = maxCities[0]
        for i in range(len(secMaxCities)):
            v = secMaxCities[i]

            # If any of max degree - second max degree cities 
            # are not directly connected
            # then answer is (maxDegree + secondMaxDegree)
            if not isConnected[u][v]:
                return maxDegree + secMaxDegree

        # If all max degree - second max degree cities 
        # are not directly connected
        return maxDegree + secMaxDegree - 1

In [ ]:
class Solution:
    def maximalNetworkRank(self, n: int, roads: List[List[int]]) -> int:
        """
        For any 2 city pair, count how many road connections they have and
        return the max pair sum (if the cities connect to each other, count
        that connection only once).

        Let the most connected city have x connections and the second
        most connected cities have (x - 1) connections
        (x - 1) + (x - 1) = 2x - 2 = highest rank not using the most connected
        assuming that they are also not connected to one another which is the
        best case scenario
        vs (x) + (x-1) - 1 = 2x - 2 = highest rank using the most connected and
        one of the 2nd most connected assuming they are connected to one another
        which is the most disfavorable scenario. If they are not connected, then
        we get 2x - 1 which is even better.

        So the greedy approach always wins, use the most connected and second
        most connected cities.

        If (x-1) is the count of the 2nd most connected cities, make sure to
        look for one that is preferably not connected to the first if we can
        find it. If not, then one connected to the first node is good enough.
        (x) + (x-1) - 1 = x + (x - 2) i.e. even if the 2nd most connected city
        is connected to the first, we're just as good as if we use the 3rd
        most connected instead of the 2nd where it is not connected to the first.

        Because the number of connections per node can be tied with others, we
        will need to consider all possiable pairs of nodes among the highest 2
        counts.
        """
        graph = {i: set() for i in range(n)}
        for A, B in roads: # Record the bi-directional road connections
            graph[A].add(B)
            graph[B].add(A)

        graph = list(sorted(graph.items(), key=lambda x: len(x[1])))
        # Consider all possiable pairs among the top 2 highest unique connection
        # counts, the global max will be somewhere in there
        uniq_counts = set()
        for node, conn in graph:
            uniq_counts.add(len(conn))
        uniq_counts = list(uniq_counts)
        uniq_counts.sort()
        if len(uniq_counts) == 1:
            min_count = uniq_counts[0]
        else: # Set the min_count to be the 2nd largest unique value we find
            min_count = uniq_counts[-2]
        
        # Drop all nodes that are not in the top 2 max counts
        graph = [x for x in graph if len(x[1]) >= min_count]
        
        # Compute the max by considering all possiable pairwise combos
        ans = 0 # Find the max rank among all node pairs
        for i in range(len(graph) - 1): # Iter over the first nodes
            for j in range(i + 1, len(graph)): # Iter of the second nodes
                rank = len(graph[i][1]) + len(graph[j][1])
                if graph[i][0] in graph[j][1]: # Don't double count
                    rank -= 1
                ans = max(ans, rank)
        return ans